# Club Stay

In [58]:
import stripe
import random

In [59]:
pub_key = "pk_test_51OkU9VGMdHo5Se5ncZOBddSt9b6rrKVnjYxQDFM4KeGNvZqEQH9oQyhiwZb1nHevn0h7ViTKWzAkyKvwhnSzNuNc00MvcGHn79"
sec_key = "sk_test_51OkU9VGMdHo5Se5nkdzsG1GttjWCkvrAWEdvIi3Yz3pnTa89SeohoTQn8lF54XLHGmPILtSxokZTCr9wNrpUC9kw007mbNBDXA"

In [60]:
stripe.api_key  = pub_key

In [61]:
def create_customer(name, surname, email, payment_method):
    
    # create fullname
    name = name + " " + surname
    
    user = stripe.Customer.create(
        name=name,
        email=email,
        payment_method=payment_method,
    )
    
    return user

In [10]:
user = create_customer("Pedro", "Souza", "pedro@email.com", cc)

NameError: name 'cc' is not defined

In [11]:
id = get_id_from_username("Vitor", "Zucher")
id

NameError: name 'get_id_from_username' is not defined

In [219]:
stripe.Customer.retrieve_payment_method(
  id,
  cc
)

<PaymentMethod payment_method id=pm_1OkXPyGMdHo5Se5nI6FiK4pM at 0x10cde58f0> JSON: {
  "billing_details": {
    "address": {
      "city": null,
      "country": null,
      "line1": null,
      "line2": null,
      "postal_code": null,
      "state": null
    },
    "email": null,
    "name": null,
    "phone": null
  },
  "card": {
    "brand": "visa",
    "checks": {
      "address_line1_check": null,
      "address_postal_code_check": null,
      "cvc_check": "pass"
    },
    "country": "US",
    "display_brand": "visa",
    "exp_month": 2,
    "exp_year": 2032,
    "fingerprint": "js54DEXLFkmx6b0C",
    "funding": "credit",
    "generated_from": null,
    "last4": "4242",
    "networks": {
      "available": [
        "visa"
      ],
      "preferred": null
    },
    "three_d_secure_usage": {
      "supported": true
    },
    "wallet": null
  },
  "created": 1708112994,
  "customer": "cus_PZgLtc1xOWFUse",
  "id": "pm_1OkXPyGMdHo5Se5nI6FiK4pM",
  "livemode": false,
  "metadata":

In [ ]:
list_users(30).data

In [327]:
stripe.api_key = sec_key

In [328]:
# check if credit card is valid
def is_valid(payment_id):
    try:
        payment_method = stripe.PaymentMethod.retrieve(payment_id)
        return payment_method["card"]["checks"]["cvc_check"] == "pass"
    except stripe.error.StripeError as e:
        print(f"Stripe API error occurred: {e}")
        return False

In [336]:
is_valid(cc)

True

In [63]:
def create_card():
    
    card = {
        "number": "4242424242424242",
        "exp_month": random.randint(1, 12),
        "exp_year": random.randint(2025, 2045),
        "cvc": random.randint(100, 1400),
    }
    
    number = card["number"]
    exp_month = card["exp_month"]
    exp_year = card["exp_year"]
    cvc = card["cvc"]
    
    return card

In [64]:
card = create_card()
card

{'number': '4242424242424242', 'exp_month': 11, 'exp_year': 2029, 'cvc': 677}

In [65]:
def create_token(card):
    token = stripe.Token.create(
        card={
            "number": card["number"],
            "exp_month": card["exp_month"],
            "exp_year": card["exp_year"],
            "cvc": card["cvc"],
        },
    )
    return token.id

In [66]:
token_id = create_token(card)
token_id

'tok_1OknIaGMdHo5Se5nWQkqfnFq'

In [68]:
stripe.api_key  = sec_key

In [78]:
payment_intent = stripe.PaymentIntent.create(
    amount=50,
    currency='usd',
    payment_method_data={
        'type': 'card',
        'card': {'token': token_id},
    },
    # confirmation_method='manual',
    confirm=True,  # Automatically try to confirm the intent
    # capture_method='manual',  # Do not capture the charge immediately
)

InvalidRequestError: Request req_BOUYHAHbZldqfy: A `return_url` must be specified because this Payment Intent is configured to automatically accept the payment methods enabled in the Dashboard, some of which may require a full page redirect to succeed. If you do not want to accept redirect-based payment methods, set `automatic_payment_methods[enabled]` to `true` and `automatic_payment_methods[allow_redirects]` to `never` when creating Setup Intents and Payment Intents.

In [73]:
payment_intent

<PaymentIntent payment_intent id=pi_3OknKjGMdHo5Se5n1MetdpkJ at 0x120b58270> JSON: {
  "amount": 50,
  "amount_capturable": 0,
  "amount_details": {
    "tip": {}
  },
  "amount_received": 0,
  "application": null,
  "application_fee_amount": null,
  "automatic_payment_methods": null,
  "canceled_at": null,
  "cancellation_reason": null,
  "capture_method": "manual",
  "client_secret": "pi_3OknKjGMdHo5Se5n1MetdpkJ_secret_KkZ6l9rCEe0xeGegYFhMtLwrX",
  "confirmation_method": "automatic",
  "created": 1708174173,
  "currency": "usd",
  "customer": null,
  "description": "Card verification charge",
  "id": "pi_3OknKjGMdHo5Se5n1MetdpkJ",
  "invoice": null,
  "last_payment_error": null,
  "latest_charge": null,
  "livemode": false,
  "metadata": {},
  "next_action": null,
  "object": "payment_intent",
  "on_behalf_of": null,
  "payment_method": null,
  "payment_method_configuration_details": null,
  "payment_method_options": {
    "card": {
      "installments": null,
      "mandate_options"

In [72]:
 print("PaymentIntent created:", payment_intent.id)

PaymentIntent created: pi_3OknKjGMdHo5Se5n1MetdpkJ


In [8]:
def create_payment_method_from_token(token_id):
    """
    Creates a payment method from a given token ID.

    :param token_id: The ID of the token created for a card.
    :return: A Stripe payment method object.
    """
    payment_method = stripe.PaymentMethod.create(
        type="card",
        card={"token": token_id},
    )
    return payment_method

In [9]:
pm = create_payment_method_from_token(token_id)

In [10]:
pm_id = pm.id
pm_id

'pm_1OkaJyGMdHo5Se5nNWv7Ylb5'

In [11]:
stripe.api_key  = sec_key

In [12]:
def create_customer(name, surname, email, payment_method):
    
    # create fullname
    name = name + " " + surname
    
    user = stripe.Customer.create(
        name=name,
        email=email,
        payment_method=payment_method,
    )
    
    return user

In [13]:
customer = create_customer("Vitor", "Zucher 3", "", pm_id)

In [14]:
customer_id = customer.id

In [15]:
customer_id

'cus_PZjnCpXiTsUYEi'

In [16]:
def attach_and_set_default_payment_method(customer_id, payment_method_id):
    # Attach the payment method to the customer
    try:
        stripe.PaymentMethod.attach(
            payment_method_id,
            customer=customer_id,
        )
    except stripe.error.StripeError as e:
        print(f"Error attaching payment method: {e}")
        return None

    # Set the attached payment method as the default for the customer
    try:
        customer = stripe.Customer.modify(
            customer_id,
            invoice_settings={'default_payment_method': payment_method_id},
        )
        print(f"Default payment method for customer {customer_id} has been updated.")
        return customer.id
    except stripe.error.StripeError as e:
        print(f"Error setting default payment method: {e}")
        return None

In [17]:
attach_and_set_default_payment_method(customer_id, pm_id)

Default payment method for customer cus_PZjnCpXiTsUYEi has been updated.


'cus_PZjnCpXiTsUYEi'

In [18]:
pm_id

'pm_1OkaJyGMdHo5Se5nNWv7Ylb5'

In [19]:
def is_valid(payment_id):
    payment_method = stripe.PaymentMethod.retrieve(payment_id)
    cvc = payment_method.card.checks.cvc_check
    
    if len(cvc) > 3 or len(cvc) < 3 or cvc == None:
        return False
    elif cvc == "pass":
        return True

In [20]:
is_valid(pm_id)

False

In [21]:
pm_id

'pm_1OkaJyGMdHo5Se5nNWv7Ylb5'

In [50]:
def list_users(pm_id):

    max = len(stripe.Customer.list(limit=100000))
    users = stripe.Customer.list(limit=max)
    users = users.data

    def get_default_payment_method(customer_id):
        customer = stripe.Customer.retrieve(customer_id)
        default_payment_method_id = customer.invoice_settings.default_payment_method
        return default_payment_method_id

    
    users_li = []

    for user in users:
        
        user = {
            "id": user.id,
            "name": user.name,
            "email": user.email,
            "payment_method": user.invoice_settings.default_payment_method,
            "valid": is_valid(pm_id)
        }
       
        users_li.append(user)
    
    return users_li

In [47]:
import pandas as pd

In [49]:
# create dataframe 
df = pd.DataFrame(users_li)
df

,id,name,email,payment_method,valid
0,cus_PZjnCpXiTsUYEi,Vitor Zucher 3,None,pm_1OkaJyGMdHo5Se5nNWv7Ylb5,False
1,cus_PZjljjFpY7AcKM,Vitor Zucher 3,None,pm_1OkaHNGMdHo5Se5n3SeRXWdC,False
2,cus_PZjfnMxrojONWO,Giovanni Italiano Massana,None,None,False
3,cus_PZjTfsZfRXRHDw,Giovanni Italiano Massana,None,None,False
4,cus_PZjGRG57PAlT76,Giovanni Italiano Massana,None,None,False
5,cus_PZj3aaw1rXcD95,Giovanni Italiano Massana,None,None,False
6,cus_PZixbcPWgyEzC4,Giovanni Italiano Massana,None,None,False
7,cus_PZitBUuaXB02Vd,Giovanni Italiano Massana,None,None,False
8,cus_PZiorliKFEiyNZ,Giovanni Italiano Massana,None,None,False
9,cus_PZinJhVRgdgof3,Giovanni Italiano Massana,None,None,False


In [55]:
def main():

    
    card = create_card()
    stripe.api_key  = pub_key
    token_id = create_token(card)
    pm = create_payment_method_from_token(token_id)
    pm_id = pm.id
    stripe.api_key = sec_key
    customer = create_customer("John", "Doe", "", pm_id)
    customer_id = customer.id
    attach_and_set_default_payment_method(customer_id, pm_id)
    valid = is_valid(pm_id)
    users_li = list_users(pm_id)
    
    
    return users_li

In [56]:
users_li = main()

Default payment method for customer cus_PZk1guVAKktri3 has been updated.


In [57]:
users_li

[{'id': 'cus_PZk1guVAKktri3',
  'name': 'John Doe',
  'email': None,
  'payment_method': 'pm_1OkaX4GMdHo5Se5nBbU6NTKh',
  'valid': False},
 {'id': 'cus_PZjnCpXiTsUYEi',
  'name': 'Vitor Zucher 3',
  'email': None,
  'payment_method': 'pm_1OkaJyGMdHo5Se5nNWv7Ylb5',
  'valid': False},
 {'id': 'cus_PZjljjFpY7AcKM',
  'name': 'Vitor Zucher 3',
  'email': None,
  'payment_method': 'pm_1OkaHNGMdHo5Se5n3SeRXWdC',
  'valid': False},
 {'id': 'cus_PZjfnMxrojONWO',
  'name': 'Giovanni Italiano Massana',
  'email': None,
  'payment_method': None,
  'valid': False},
 {'id': 'cus_PZjTfsZfRXRHDw',
  'name': 'Giovanni Italiano Massana',
  'email': None,
  'payment_method': None,
  'valid': False},
 {'id': 'cus_PZjGRG57PAlT76',
  'name': 'Giovanni Italiano Massana',
  'email': None,
  'payment_method': None,
  'valid': False},
 {'id': 'cus_PZj3aaw1rXcD95',
  'name': 'Giovanni Italiano Massana',
  'email': None,
  'payment_method': None,
  'valid': False},
 {'id': 'cus_PZixbcPWgyEzC4',
  'name': 'Giovan